<a href="https://colab.research.google.com/github/yagoandres/dea_studies/blob/main/julia_malmquist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
using Pkg; Pkg.add("DataEnvelopmentAnalysis")
import Pkg; Pkg.add("XLSX")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Xorg_libpciaccess_jll ─── v0.18.1+0
   Installed MUMPS_seq_jll ─────────── v500.800.100+0
   Installed Hwloc_jll ─────────────── v2.12.2+0
   Installed OpenBLAS32_jll ────────── v0.3.29+0
   Installed BenchmarkTools ────────── v1.6.0
   Installed MutableArithmetics ────── v1.6.5
   Installed MathOptInterface ──────── v1.45.0
   Installed Ipopt_jll ─────────────── v300.1400.1900+0
   Installed GLPK_jll ──────────────── v5.0.1+1
   Installed METIS_jll ─────────────── v5.1.3+0
   Installed CodecBzip2 ────────────── v0.8.5
   Installed StructTypes ───────────── v1.11.0
   Installed JSON3 ─────────────────── v1.14.3
   Installed DataEnvelopmentAnalysis ─ v0.9.1
   Installed ASL_jll ───────────────── v0.1.3+0
   Installed SPRAL_jll ─────────────── v2025.5.20+0
   Installed GLPK ──────────────────── v1.2.1
   Installed JuMP ──────────────────── v1.29.1
   Installed XML2_jll ──────────────

In [7]:
using DataEnvelopmentAnalysis
using XLSX
file_path = "/content/20250914_DEA_EnergySecurityPaper2.xlsx"
xf = XLSX.readxlsx(file_path)


XLSXFile("/content/20250914_DEA_EnergySecurityPaper2.xlsx") containing 10 Worksheets
            sheetname size          range        
-------------------------------------------------
                   EO 18x2          A1:B18       
I1-Installed Capacit… 48x25         A1:Y48       
  Calculations of HHI 828x40        A1:AN828     
I2-Energy diversific… 71x39         A1:AM71      
     I3-Energy import 48x39         A1:AM48      
        I4-%Renewable 58x39         A1:AM58      
        O1-MWh capita 21x26         A1:Z21       
        O2-GDP_capita 21x26         A1:Z21       
   O3-Electrification 21x26         A1:Z21       
  O4-carbon intensity 25x26         A1:Z25       


In [8]:
# Get the list of sheet names
sheet_names = XLSX.sheetnames(xf)

println("Checking sheet names:")
i_count = 0
o_count = 0

for sheet_name in sheet_names
    # Convert sheet name to lowercase for case-insensitive comparison
    lower_sheet_name = lowercase(sheet_name)

    if startswith(lower_sheet_name, "i")
        println("'$(sheet_name)' starts with 'I'")
        i_count += 1
    elseif startswith(lower_sheet_name, "o")
        println("'$(sheet_name)' starts with 'O'")
        o_count += 1
    else
        println("'$(sheet_name)' does not start with 'I' or 'O'")
    end
end

println("\nNumber of sheets starting with 'I':  $i_count")
println("Number of sheets starting with 'O':   $o_count")

Checking sheet names:
'EO' does not start with 'I' or 'O'
'I1-Installed Capacity Capita' starts with 'I'
'Calculations of HHI' does not start with 'I' or 'O'
'I2-Energy diversification HHI' starts with 'I'
'I3-Energy import' starts with 'I'
'I4-%Renewable' starts with 'I'
'O1-MWh capita' starts with 'O'
'O2-GDP_capita' starts with 'O'
'O3-Electrification' starts with 'O'
'O4-carbon intensity' starts with 'O'

Number of sheets starting with 'I':  4
Number of sheets starting with 'O':   4


In [9]:
X = Array{Float64,3}(undef, 20, i_count, 24);
# Does not consider Electrification
Y = Array{Float64,3}(undef, 20, o_count-1, 24);



In [18]:
function normalize_data(data)
    """
    Normalize data using robust scaling
    """
    min_vals = minimum(data, dims=1)
    max_vals = maximum(data, dims=1)
    range_vals = max_vals - min_vals
    #println(range_vals)
    if any(range_vals .== 0)
        println("ERROR range_vals=0")
    end
    # range_vals[range_vals .== 0] .= 1 # Uncomment and modify if you need to handle zero ranges

    # Scale to [0.1, 1] range
    normalized = 0.1 .+ 0.9 .* (data .- min_vals) ./ range_vals
    return normalized
end

normalize_data (generic function with 1 method)

In [30]:
sh=xf["I2-Energy diversification HHI"]
input_installed_capacity_capita=sh["B2:Y21" ]
#sh=xf[sheet_name]
table=sh["B2:Y21" ]
println(table[:,1])
input_installed_capacity_capita = 1 ./ input_installed_capacity_capita
table = 1 ./ table
println("\nElement-wise inverse of input_installed_capacity_capita:")
println(input_installed_capacity_capita_inverse)

Any[4114.3151339999995, 7029.982510000001, 7649.75491, 4047.171753, 6400.3657939999985, 6260.104388000001, 3710.5159100000005, 4975.500104000001, 2626.1350079999993, 2925.1148359999997, 2201.5143620000003, 3018.97641, 2769.581916, 5031.505922, 8700.491913999997, 3205.611601, 3005.8575220000002, 3080.0505919999996, 3392.8632410000005, 2419.386806]

Element-wise inverse of input_installed_capacity_capita:
[0.00024305381756885134 0.00024897344978875566 0.0002406772670231624 0.00024340772055017783 0.0002476912591229905 0.0002565078261336775 0.000270417596042255 0.00026928675607057885 0.0002688321507905255 0.00028246598204202124 0.000286394623834947 0.00027863881218746447 0.000260514455584374 0.0002716604999271361 0.0002635587139273346 0.00026883720831875647 0.00025450199548855717 0.00022271036320100227 0.00021034664096574005 0.00021141273194333145 0.00024080010322811464 0.0002621906793923479 0.00030482224623576653 0.00030361973363609576; 0.0001422478645683003 0.0001513735219985056 0.000162

In [39]:
println("Checking sheet names:")
i_count = 0
o_count = 0

for sheet_name in sheet_names
    # Convert sheet name to lowercase for case-insensitive comparison
    lower_sheet_name = lowercase(sheet_name)
    sh=xf[sheet_name]
    table=sh["B2:Y21" ]
    if startswith(sheet_name,"EO")
      continue
    end
    if startswith(sheet_name,"O3-Electrification")
      continue
    end
    if startswith(lower_sheet_name, "i")
        println("$(sheet_name) starts with 'I'")

        if startswith(lower_sheet_name, "i2")
           println("i2: inverse")
           table = 1 ./ table
        end

        if startswith(lower_sheet_name, "i3")
           println("i3: -1")
           table = -table
        end
        if size(X, 1) == size(table, 1) # and size(X, 3) == size(input_rd_expenditure, 2)
             # Iterate through the third dimension of X and assign columns
              for k in 1:size(X, 3)
                #println("k $(k)")
                X[:, i_count+1, k] = normalize_data(table[:, k])
              end
              println("Successfully assigned columns to X.")
        else
          println("Dimension mismatch: Cannot assign input_rd_expenditure to X.")
          println(size(X,1))
          println(size(table,1))
          println(size(X,3))
          println(size(table,2))
        end # Added missing 'end' for the inner if block
        i_count += 1
    elseif startswith(lower_sheet_name, "o")
        println("$(sheet_name) starts with 'O'")

        if startswith(lower_sheet_name, "o4")
           println("o4: inverse")
           table = 1 ./ table
        end

        if size(Y, 1) == size(table, 1) # and size(X, 3) == size(input_rd_expenditure, 2)
             # Iterate through the third dimension of X and assign columns
              for k in 1:size(Y, 3)
                Y[:, o_count+1, k] = normalize_data(table[:, k])
              end
              println("Successfully assigned columns to Y.")
        else
          println("Dimension mismatch: Cannot assign input_rd_expenditure to Y.")
          println(size(Y,1))
          println(size(table,1))
          println(size(Y,3))
          println(size(table,2))
        end # Added missing 'end' for the inner if block


        o_count += 1
    else
        println("'$(sheet_name)' does not start with 'I' or 'O'")
    end
end

Checking sheet names:
I1-Installed Capacity Capita starts with 'I'
Successfully assigned columns to X.
'Calculations of HHI' does not start with 'I' or 'O'
I2-Energy diversification HHI starts with 'I'
i2: inverse
Successfully assigned columns to X.
I3-Energy import starts with 'I'
i3: -1
Successfully assigned columns to X.
I4-%Renewable starts with 'I'
Successfully assigned columns to X.
O1-MWh capita starts with 'O'
Successfully assigned columns to Y.
O2-GDP_capita starts with 'O'
Successfully assigned columns to Y.
O4-carbon intensity starts with 'O'
o4: inverse
Successfully assigned columns to Y.


In [40]:
println("Printing values in X:")
for j in 1:size(X, 2)
    for k in 1:size(X, 3)
        println("X[:, $(j), $(k)]:")
        println(X[:, j, k])
    end
end

Printing values in X:
X[:, 1, 1]:
[0.2579302587176603, 0.5479752530933634, 0.1751687289088864, 1.0, 0.1410011248593926, 0.5188695163104612, 0.42370641169853773, 0.1, 0.103037120359955, 0.4021934758155231, 0.5284870641169853, 0.20199662542182228, 0.43079302587176604, 0.9296400449943757, 0.296400449943757, 0.3697975253093363, 0.18681102362204724, 0.40649606299212593, 0.79322272215973, 0.4130764904386951]
X[:, 1, 2]:
[0.2629245018243054, 0.5463373561605389, 0.1747684535503789, 1.0, 0.14319393769295538, 0.520319955094022, 0.4263541959023295, 0.1, 0.10252596126859388, 0.4033679483581252, 0.5296660117878192, 0.20129104687061466, 0.4339320797081112, 0.9275049115913556, 0.29500420993544774, 0.3748245860230143, 0.18790345214706708, 0.405893909626719, 0.7969127140050519, 0.4157451585742352]
X[:, 1, 3]:
[0.26016102165463634, 0.540505274847307, 0.17370905052748475, 1.0, 0.14547473625763466, 0.5180177679067184, 0.4258189894503054, 0.1, 0.10249861188228762, 0.4020821765685729, 0.5277623542476402, 0.

In [41]:
println("Printing values in Y:")
for j in 1:size(Y, 2)
    for k in 1:size(Y, 3)
        println("Y[:, $(j), $(k)]:")
        println(Y[:, j, k])
    end
end

Printing values in Y:
Y[:, 1, 1]:
[0.19156312642635914, 0.6128403962910877, 0.1731314240839747, 0.9999999999999999, 0.1289444946325449, 0.5006469394800598, 0.4064907617924408, 0.10407184454823805, 0.1, 0.30079013533314736, 0.48668830308354005, 0.17593346880659258, 0.3606260745568435, 0.4827284090648817, 0.2891883114759838, 0.37137293504985924, 0.16822649712166654, 0.3795760724790159, 0.7153303126336846, 0.36766290859275275]
Y[:, 1, 2]:
[0.2029441537294345, 0.6443400562593983, 0.16765247736991784, 1.0, 0.13303975601363976, 0.5205776149455774, 0.4228686816754903, 0.10290429002538504, 0.1, 0.30899521312572586, 0.49377098619262283, 0.17925692209393818, 0.3754665127598701, 0.49678557079932595, 0.2929151741595434, 0.39937800681950686, 0.16675455533417294, 0.39460630479337544, 0.719490508455476, 0.38516439982490913]
Y[:, 1, 3]:
[0.2063617175603088, 0.629789027719347, 0.16942027719727853, 0.9999999999999999, 0.13774516376581689, 0.519527278521004, 0.42109880006612743, 0.10225812909412164, 0.1,

In [42]:
malmquist(X, Y)

ArgumentError: ArgumentError: colnms should have length 0 or 69